In [2]:
import ctypes
class Array:
    def __init__(self, size):
        assert size > 0
        self.size = size
        PyArrayTypes = ctypes.py_object * size
        self._elements = PyArrayTypes()
        self.clear(None)
        
    def length(self):
         return self.size
        
    def __getitem__ (self, index):
        assert index >= 0 and index < self.size
        return self._elements[index]
        
    def __setitem__ (self, index, value):
        assert index >=0 and index < self.size
        self._elements[index] = value 
        
    def clear(self, value):
        for i in range(self.size):
            self._elements[i] = value
                
    def __iter__ (self):
        return _ArrayIterator( self._elements )
    
    def __str__ (self):
        s = "["
        for x in range(self.length()):
            if x != self.length() -1:
                s+= str(self.__getitem__(x)) +','
            else:
                s+= str(self.__getitem__(x))
        s += "]"
        return s
            
        
class _ArrayIterator:
    def __init__(self, thearray):
        self._arrayref = thearray
        self._curNdx = 0
    
    def __iter__ (self):
        return _ArrayIterator(array._elements)
    
    def __next__(self):
        if self._curNdx < len(self._arrayref):
            entry = self._arrayref[self._curNdx]
            self._curNdx += 1
            return entry
        else:
            raise StopIteration
            
class array2d:
    def __init__(self, row, col):
        self.row = row
        self.col = col
        self.rowarray = Array(row)
        for i in range(row):
            self.rowarray[i] = Array(col)
        self.clear(None)
            
    def numrows(self):
        return (self.row)
    
    def numcols(self):
        return (self.rowarray[0].length())
    
    def clear(self, value):
        for i in range(self.numrows()):
            self.rowarray[i].clear(value)  
    
    def setitem(self, indexlist, value):
        assert len(indexlist)==2
        assert indexlist[0] >=0 and indexlist[0] < self.numrows()
        assert indexlist[1] >=0 and indexlist[1] < self.numcols()
        onedarray = self.rowarray[indexlist[0]]
        onedarray[indexlist[1]] = value
        
    def getitem(self, indexlist):
        assert len(indexlist)==2
        assert indexlist[0] >=0 and indexlist[0] < self.numrows()
        assert indexlist[1] >=0 and indexlist[1] < self.numcols()
        onedarray = self.rowarray[indexlist[0]]
        return onedarray[indexlist[1]]
    
    def __str__(self):
        s = "["
        for i in range(self.numrows()):
            s += "["
            for j in range(self.numcols()):
                l = [i,j]
                if j != self.numcols() -1:
                    s += str(self.getitem(l)) +","
                else:
                    s += str(self.getitem(l))
            s += "]"
        s += "]"
        return s

# Task 1

In [8]:
class GrayScale:
    def __init__ (self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.pixels = array2d(rows, cols)
        self.clear(0)
        
    def width(self):
        return self.cols
    
    def height(self):
        return self.rows
    
    def clear(self, value):
        if value < 0:
            value = 0
        if value > 255:
            value = 255
        self.pixels.clear(value)
    
    def getitem(self, indexlist):
        return self.pixels.getitem(indexlist)
    
    def setitem(self, indexlist, value):
        if value < 0:
            value = 0
        if value > 255:
            value = 255
        self.pixels.setitem(indexlist, value)
        
image1 = GrayScale(480, 720)
print(image1.width())
print(image1.height())
image1.clear(234)
print(image1.getitem([234, 321]))
image1.setitem([456, 678], 102)

720
480
234


# Task 2

In [18]:
import random
class Matrix:
    def __init__(self, row, col):
        self.matrix = array2d(row, col)
        self.matrix.clear(None)
        
    def numrows(self):
        return self.matrix.numrows()
        
    def numcols(self):
        return self.matrix.numcols()
    
    def getitem(self, indexlist):
        return self.matrix.getitem(indexlist)
    
    def setitem(self, indexlist, value):
        self.matrix.setitem(indexlist, value)
        
    def scaleBy(self, value):
        for x in range(self.numrows()):
            for y in range(self.numcols()):
                self.matrix[x][y] = self.matrix[x][y] * value
        
    
    def add(self, matrix):
        assert self.numrows() == matrix.numrows()
        assert self.numcols() == matrix.numcols()
        newmatrix = Matrix(self.numrows(), self.numcols())
        for x in range(self.numrows()):
            for y in range(self.numcols()):
                l = [x,y]
                val = self.getitem(l) + matrix.getitem(l)
                newmatrix.setitem(l, val) 
        return newmatrix
    
    def subtract(self, matrix):
        assert self.numrows() == matrix.numrows()
        assert self.numcols() == matrix.numcols()
        newmatrix = Matrix(self.numrows(), self.numcols())
        for x in range(self.numrows()):
            for y in range(self.numcols()):
                l = [x,y]
                val = self.getitem(l) - matrix.getitem(l)
                newmatrix.setitem(l, val)
        return newmatrix
    
    def transpose(self):
        newmatrix = Matrix(self.numcols(), self.numrows())
        for x in range(self.numrows()):
            for y in range(self.numcols()):
                normal = [x,y]
                l = [y,x]
                val = self.getitem(normal)
                newmatrix.setitem(l, val)
        return newmatrix
    
    def multiply(self, matrix):
        row = self.numrows()
        col = matrix.numcols()
        assert self.numcols() == matrix.numrows()
        newmatrix = Matrix(self.numrows(), matrix.numcols())
        for x in range(row):
            for y in range(col):
                current = 0
                newindex = [x,y]
                for z in range(self.numcols()):
                    first = [x,z]
                    second = [z,y]
                    current += self.getitem(first) * matrix.getitem(second)
                newmatrix.setitem(newindex, current)
        return newmatrix
    
    def scale(self, value):
        for row in range(self.numrows()):
            for col in range(self.numcols()):
                l = [row, col]
                current = self.getitem(l)
                new = current * value
                self.setitem(l, new)
                #self.matrix[row][col] = self.matrix[row][col] * value
        
    
    def __str__(self):
        s = "["
        for i in range(self.numrows()):
            for j in range(self.numcols()):
                l = [i,j]
                if j != self.numcols() -1 :
                    s += str(self.getitem(l)) + ","
                else:
                    s += str(self.getitem(l))
            if i != self.numrows() -1:
                s += "\n"
        s += "]"
        return s

#Matrix Setup
obj3 = Matrix(2, 3)
obj5 = Matrix(2, 3)
obj4 = Matrix(3, 2)
for x in range(obj3.numrows()):
    for y in range(obj3.numcols()):
        l = [x,y]
        obj3.setitem(l, random.randint(1, 10))

for x in range(obj4.numrows()):
    for y in range(obj4.numcols()):
        l = [x,y]
        obj4.setitem(l, random.randint(1, 10))

for x in range(obj5.numrows()):
    for y in range(obj5.numcols()):
        l = [x,y]
        obj5.setitem(l, random.randint(1, 10))

#Matrix Test
print(obj3, "\n")
print(obj5, "\n")
print(obj4, "\n")
print(obj3.add(obj5), "\n")
print(obj5.subtract(obj3), "\n")
print(obj3.multiply(obj4), "\n")
print(obj3.transpose(), "\n")
obj3.scale(10)
print(obj3)

[4,9,7
8,5,5] 

[8,10,1
6,10,8] 

[6,2
8,10
2,8] 

[12,19,8
14,15,13] 

[4,1,-6
-2,5,3] 

[36,40
34,38] 

[4,8
9,5
7,5] 

[40,90,70
80,50,50]


# Task 3, 4, and 5

In [9]:
class LifeGrid:
    alive = 1
    dead = 0
    def __init__ (self, rows, cols):
        self.grid = array2d(rows, cols)
        self.configure([])
        
    def numrows(self):
        return self.grid.numrows()
    
    def numcols(self):
        return self.grid.numcols()
    
    def configure(self, indexlist):
        for row in range(self.grid.numrows()):
            for col in range(self.grid.numcols()):
                self.clearcell(row, col)
        
        for index in indexlist:
            self.setcell(index[0], index[1])
    
    def clearcell(self, row, col):
        l = [row, col]
        self.grid.setitem(l, self.dead)
    
    def setcell(self, row, col):
        l = [row, col]
        self.grid.setitem(l, self.alive)
    
    def isalive(self, row, col):
        l = [row, col]
        return self.grid.getitem(l)
    
    def numlivecells(self, row, col):
        l = [row, col]
        count = 0
        if row != 0:
            if self.isalive(row-1, col):
                count += 1
        if row != self.numrows() -1:
            if self.isalive(row+1, col):
                count += 1
        if col != 0:
            if self.isalive(row, col-1):
                count += 1
        if col != self.numcols() -1:
            if self.isalive(row, col+1):
                count += 1
                
        if row != 0 and col != 0:
            if self.isalive(row-1, col-1):
                count += 1
        
        if row != 0 and col != self.numrows() -1:
            if self.isalive(row-1, col+1):
                count += 1
        
        if row != self.numrows()-1 and col != 0:
            if self.isalive(row+1, col-1):
                count += 1
        
        if row != self.numrows()-1 and col != self.numcols()-1:
            if self.isalive(row+1, col+1):
                count += 1
        return count
    
    def __str__(self):
        s =""
        for i in range(self.numrows()):
            for j in range(self.numcols()):
                l = [i, j]
                if self.grid.getitem(l) == 0:
                    s +=  ".  "
                else:
                    s += "@  "
            s += "\n"
        return s
    
    def nextgrid(self):
        livecells = []
        for i in range(self.numrows()):
            for j in range(self.numcols()):
                l = [i, j]
                if self.grid.getitem(l):
                    if self.numlivecells(i, j) == 2 or self.numlivecells(i, j) == 3:
                        livecells.append(l)
                else:
                    if self.numlivecells(i, j) == 3:
                        livecells.append(l)
        return livecells
    
        

row = int(input("Enter the rows of the grid: "))
col = int(input("Enter the columns of the grid: "))
generations = int(input("Enter the number of generations: "))
game = LifeGrid(row, col)
game.setcell(5, 5)
game.setcell(6, 5)
game.setcell(7, 4)
game.setcell(7, 6)
game.setcell(8, 5)
game.setcell(9, 5)
for i in range(1, generations + 1):
    print("Generation:", i)
    print("-"*50)
    game.configure(game.nextgrid())
    print(game)
    print("-"*50)

Enter the rows of the grid: 20
Enter the columns of the grid: 20
Enter the number of generations: 5
Generation: 1
--------------------------------------------------
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  @  @  @  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  @  .  @  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  @  @  @  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  .  .  